In [ ]:
from processing_utils import train_test_split, target_encoding, X_y_split


import polars as pl
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import cross_val_score, GridSearchCV

import matplotlib.pyplot as plt
import plotly

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


import optuna
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
import numpy as np


In [2]:
df: pl.DataFrame = pl.read_parquet(r'C:\Users\310\Desktop\Progects_Py\data\microstructure_price_prediction_data\cross_section\df_cross_section_V0.1_.parquet')

In [3]:
df_train, df_test = train_test_split(df, train_test_ratio=0.01, print_stat=True)

Train len for AVAX-USDT is 13536
Test len for AVAX-USDT is 1340121
Train len for DOGE-USDT is 32493
Test len for DOGE-USDT is 3216827


In [4]:
target_var: str = "target_five_step_ahead"

In [5]:
df_train, df_test = target_encoding(target_var, df_train, df_test)

In [6]:
df_train = df_train.select(pl.exclude('trade_time', 'is_buyer_maker', 'date', 'target'))
df_test = df_test.select(pl.exclude('trade_time', 'is_buyer_maker', 'date', 'target'))

In [7]:
X_train, y_train, X_test, y_test = X_y_split(target_var, df_train, df_test)
print(y_train.shape)


(46013,)


In [8]:
def objective(trial):
    # Hyperparameter space
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int("max_depth", 3, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_weight_fraction_leaf = trial.suggest_float("min_weight_fraction_leaf", 0.0, 0.5)
    min_impurity_decrease = trial.suggest_float("min_impurity_decrease", 0.0, 0.5)

    # Define the RandomForestRegressor with trial parameters
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_weight_fraction_leaf=min_weight_fraction_leaf,
        min_impurity_decrease=min_impurity_decrease,
        random_state=42,
        n_jobs=-1,
    )

    # Use K-Fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mse_scores = -cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv=kf)

    # Return the mean MSE score
    return np.mean(mse_scores)

# Set up the Optuna study
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50, show_progress_bar=True)

# Print the best parameters
print("Best hyperparameters:", study.best_params)
print("Best MSE:", study.best_value)

# Visualize optimization progress
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_parallel_coordinate(study).show()

# Launch Optuna dashboard (optional)
# Uncomment the line below to launch Optuna's dashboard
!optuna-dashboard sqlite:///optuna.db --port 8080


[I 2024-11-23 21:23:54,968] A new study created in memory with name: no-name-2cb77987-c5b1-4bea-90c0-82643b86108f


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-11-23 21:24:25,874] Trial 0 finished with value: 71.03770017160303 and parameters: {'n_estimators': 278, 'max_depth': 37, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.402823561786303, 'min_impurity_decrease': 0.01552328163044181}. Best is trial 0 with value: 71.03770017160303.
[I 2024-11-23 21:24:38,931] Trial 1 finished with value: 0.00479086148898065 and parameters: {'n_estimators': 83, 'max_depth': 4, 'min_samples_split': 8, 'min_weight_fraction_leaf': 0.17584563643814521, 'min_impurity_decrease': 0.4381604176395775}. Best is trial 1 with value: 0.00479086148898065.
[I 2024-11-23 21:25:11,632] Trial 2 finished with value: 69.80232533470048 and parameters: {'n_estimators': 329, 'max_depth': 12, 'min_samples_split': 6, 'min_weight_fraction_leaf': 0.4004213643326946, 'min_impurity_decrease': 0.2742907800571154}. Best is trial 1 with value: 0.00479086148898065.
[I 2024-11-23 21:25:29,346] Trial 3 finished with value: 0.004790864400844805 and parameters: {'n_estimators'

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.